In [226]:
import pandas as pd
import numpy as np
import os, json

import matplotlib.pyplot as plt
from tabulate import tabulate
from collections import defaultdict, Counter
%matplotlib inline

plt.rcParams['font.family'] = 'serif'
plt.rcParams["font.serif"] = ["Nimbus Roman"]

**Load all data: targeting, participant -> ads maps, codes, surveys**

In [ ]:
targeting_params = pd.read_csv('/home/piotr/targeting_params.csv', low_memory=False)
params_df = targeting_params.set_index('adid')

import json
interests = json.loads(open('/home/piotr/interest_lookup.json').read())

In [228]:
DATA_DIR = 'data/'
active_pids = open(os.path.join(DATA_DIR, 'pid_active_contribs_cleaned.txt'), 'r').readlines()
active_pids = [p.strip() for p in active_pids]

# load PID to ADID mapping of annotated ads
pid_adid_files = [os.path.join(DATA_DIR, f'surveys/survey-{n}/survey{n}_coding_subsample.tsv') for n in range(1, 8)]
pid_adid_files.append(os.path.join(DATA_DIR, 'surveys/survey-1/survey1_batch2_coding_subsample.tsv'))

# Load with frequency too
pid_adid_freqs_noind = pd.read_csv(os.path.join(DATA_DIR, 'participant_ad_freqs.tsv'), sep='\t')
pid_adid_freqs = pid_adid_freqs_noind.set_index(['pid', 'adid'])

pid_adid = pd.read_csv(pid_adid_files[0], sep='\t')
for f in pid_adid_files[1:]:
    pid_adid = pd.concat([pid_adid, pd.read_csv(f, sep='\t')], ignore_index=True)    

# only keep PID ADID mapping for active participants
pid_adid = pid_adid[pid_adid['pid'].isin(active_pids)]
print('pid_adid shape:', pid_adid.shape)  

codes = pd.read_csv(os.path.join(DATA_DIR, 'ad_codes.tsv'), sep='\t')
print('codes shape:', codes.shape)

adid_codes = dict(zip(codes['adid'], codes['codes']))

pid_adid shape: (35387, 2)
codes shape: (32586, 2)


In [133]:
survey_file = 'survey_responses.tsv'
survey = pd.read_csv(os.path.join(DATA_DIR, survey_file), sep='\t')
survey_codes = survey.merge(codes, 'inner', 'adid')

short_names = {
    # dislike
    "I do not dislike this ad.": "dont-dislike",
    "I cannot tell what is being advertised.": "unclear",
    "It is irrelevant to me, or doesn’t contain interesting information.": "irrelevant",
    "I find the ad pushy or it causes me to feel anxious.": "pushy",
    "It contains clickbait, sensationalized, or shocking content.": "clickbait",
    "I do not trust this ad, it seems like a scam.": "scam",
    "I dislike the type of product being advertised.": "dislike-product",
    "I do not like the design of the ad.": "dislike-design",
    "I find the content uncomfortable, offensive, or repulsive.": "uncomfortable",    
    "I dislike the advertiser.": "dislike-advertiser",
    "I dislike the political nature of the ad.": "political",
    # like
    "The content is engaging, clever or amusing.": "amusing",
    "It is well designed or eye-catching.": "like-design",
    "I am interested in what is being advertised.": "interested",
    "It is clear what product the ad is selling.": "clear",
    "I trust the ad, it looks authentic or trustworthy.": "trust-ad",
    "I trust the advertiser.": "trust-advertiser",
    "It is useful, interesting, or informative.": "useful",
    "It clearly looks like an ad and can be filtered out.": "filterable",
    "I do not like this ad.": "dont-like"
}

like_list = ["amusing", "like-design", "interested", "clear", "trust-ad",
     "trust-advertiser", "useful", "filterable", "dont-like"]
dislike_list = ["dont-dislike", "unclear", "irrelevant", "pushy", "clickbait",
    "scam", "dislike-product", "dislike-design", "uncomfortable",
    "dislike-advertiser", "political"]

# actual reasons for liking or disliking
like_reasons = set(like_list) - set(["dont-like"])
dislike_reasons = set(dislike_list) - set(["dont-dislike"])

### Compute Top 20% Consumers

In [229]:
DONT_COUNT = ["Can't determine, return to this one", "Study", "UNCAT", 
             "Political", "CA Lawsuit"]

# NOTE: to count for groups of codes, add another parameters
def measure_exposure(pids: list, adid_codes: dict,
                     pid_adid: pd.DataFrame, freqs: pd.DataFrame, code_groups: dict = None) -> dict:
    # map from PID -> Code -> count (frequency included)
    exposure = {p: Counter() for p in pids}
    total_ads = {}    # map from PID to total number of ads
    for p in pids:
        pid_ads = pid_adid[pid_adid['pid'] == p]
        total_ads[p] = 0
        for aid in pid_ads['adid']:            
            f = int(freqs.loc[p, str(aid)])
            if int(aid) in adid_codes:
                allcodes = adid_codes[int(aid)].split(';')                
                if not code_groups:
                    for c in allcodes:
                        if c not in DONT_COUNT:
                            # code c was seen f times by participant p
                            exposure[p][c] += f                
                            total_ads[p] += f
                else:
                    # grouped code counting from `code_groups` in the else clause, don't double-count
                    done = set([])
                    for c in allcodes:
                        if c in code_groups and c not in DONT_COUNT:
                            mapped = code_groups[c]
                            if mapped not in done:
                                # code group mapped was see f times by participant p
                                exposure[p][mapped] += f
                                total_ads[p] += f
                                done.add(mapped)
            else:
                # we don't have annotations for this, don't count in analysis
                # total_ads[p] -= 1
                continue                        
            
    # NOTE: total_ads contains not just number of ads, but number of observations, factoring in frequencies
    return exposure, total_ads

exposure, ann_ads_count = measure_exposure(active_pids, adid_codes, pid_adid, pid_adid_freqs)

# count grouped exposure
harmful, non_harmful = 'harmful', 'non_harmful'
groups = {
    'Potentially Harmful': harmful, 'Sensitive': harmful, 'Financial': harmful,
    'Clickbait': harmful, 'Potentially Prohibited': harmful, 'Benign': non_harmful,
    'Opportunity': 'other', 'Healthcare': 'other'
}
grouped_exposure, _ = measure_exposure(active_pids, adid_codes, pid_adid, pid_adid_freqs, groups)

In [230]:
total_vals = [ann_ads_count[p] for p in active_pids]

neutral_counts = [grouped_exposure[p][non_harmful] for p in active_pids]
neutral_expos_fracs = np.array(neutral_counts) / np.array(total_vals)

harmful_counts = [grouped_exposure[p][harmful] for p in active_pids]
harmful_expos_fracs = np.array(harmful_counts) / np.array(total_vals)

other_counts = [grouped_exposure[p]['other'] for p in active_pids]
other_expos_frac = np.array(other_counts) / np.array(total_vals)

# cut off THRESH%
order = [(p, grouped_exposure[p][harmful] / ann_ads_count[p]) for p in active_pids if ann_ads_count[p] > 0]
order = sorted(order, key=lambda t: t[1])
order = [t[0] for t in order]
THRESH = 0.2
thresh = int(THRESH * len(order))
top_consumers = order[-thresh:]

# filter out top consumers who didn't fill out surveys
# top_consumers_filtered = set(top_consumers) & set(survey['pid'].unique())
top_consumers_filtered = [p for p in top_consumers if p in survey['pid'].unique()]
print(len(top_consumers_filtered))

19


<ipython-input-230-3c26d311b593>:4: RuntimeWarning: invalid value encountered in divide
  neutral_expos_fracs = np.array(neutral_counts) / np.array(total_vals)
<ipython-input-230-3c26d311b593>:7: RuntimeWarning: invalid value encountered in divide
  harmful_expos_fracs = np.array(harmful_counts) / np.array(total_vals)
<ipython-input-230-3c26d311b593>:10: RuntimeWarning: invalid value encountered in divide
  other_expos_frac = np.array(other_counts) / np.array(total_vals)


In [272]:
# what fraction of their problematic ads do these people dislike?? judge via dont-like
def count_dislike(pid, qstring):
    filtered = survey_codes[(survey_codes['pid'] == pid) & (survey_codes.codes.str.contains(qstring))]

    disliked = 0
    for i in range(filtered.shape[0]):    
        row = filtered.iloc[i]
        if 'Study' in row['codes']:
            continue

        if "I do not like this ad." in row['like']:
            disliked += 1

    return disliked, filtered.shape[0]

In [279]:
# show table
table = []
tab_codes = ['Clickbait', 'Potentially Harmful', 'Potentially Prohibited', 'Financial', 'Sensitive']
prob_qstring = 'Clickbait|Potentially Prohibited|Potentially Harmful|Sensitive|Financial'
neut_qstring = 'Benign'

# median row
row = ['Median', '', '']
# active_pids.remove('372771')
for c in tab_codes:
    row.append(int(np.median([exposure[p][c] for p in active_pids if ann_ads_count[p] >= 100])))
table.append(row)
table.append(['----------']*6)

i = 1
for p in top_consumers_filtered:
    if ann_ads_count[p] >= 100:
        disliked, tot = count_dislike(p, prob_qstring)
        disliked_neutral, tot_neutral = count_dislike(p, neut_qstring)
        
        latex_row = ["\\textbf{" + f'P{i}' + '}', f'{grouped_exposure[p][harmful] / ann_ads_count[p]:.2f}']
        row = [f'({i}) {p[:5]} ({grouped_exposure[p][harmful] / ann_ads_count[p]:.2f})',
               # disliked problematic, disliked neutral
               f'{disliked/tot:.2f}', f'{disliked_neutral/tot_neutral:.2f}']
        for c in tab_codes:
            row.append(exposure[p][c])
            latex_row.append(str(exposure[p][c]))
        table.append(row)
        print(' & '.join(latex_row) + ' \\\\')
        i += 1
        
print(tabulate(table, tablefmt='github',
      headers=['', 'disliked Prob.', 'disliked Neutral', 'Clickbait', 'Deceptive', 'Prohibited', 'Financial', 'Sensitive']))

# TODO: I think this is in good shape now, I should generate a LaTeX version of this table and move it to the paper

\textbf{P1} & 0.17 & 5 & 4 & 1 & 2 & 25 \\
\textbf{P2} & 0.17 & 44 & 6 & 2 & 80 & 5 \\
\textbf{P3} & 0.17 & 53 & 4 & 8 & 66 & 8 \\
\textbf{P4} & 0.18 & 39 & 0 & 0 & 2 & 0 \\
\textbf{P5} & 0.18 & 11 & 8 & 5 & 46 & 58 \\
\textbf{P6} & 0.20 & 4 & 1 & 0 & 14 & 1 \\
\textbf{P7} & 0.20 & 8 & 1 & 0 & 24 & 6 \\
\textbf{P8} & 0.22 & 21 & 16 & 2 & 42 & 14 \\
\textbf{P9} & 0.22 & 9 & 6 & 0 & 16 & 19 \\
\textbf{P10} & 0.23 & 46 & 18 & 11 & 130 & 14 \\
\textbf{P11} & 0.26 & 6 & 0 & 1 & 36 & 3 \\
|                   | disliked Prob.   | disliked Neutral   | Clickbait   | Deceptive   | Prohibited   |   Financial |   Sensitive |
|-------------------|------------------|--------------------|-------------|-------------|--------------|-------------|-------------|
| Median            |                  |                    | 6           | 4           | 2            |          14 |           5 |
| ----------        | ----------       | ----------         | ----------  | ----------  | ----------   |         

In [270]:
# median fraction of dislike?
dislike_frac = []
for p in active_pids:
    if ann_ads_count[p] >= 100:
        d, t = problematic_dislike(p)
        if t > 0:
            dislike_frac.append(d/t)
        
print('median dislike frac:', np.median(dislike_frac))

median dislike frac: 0.45454545454545453


In [236]:
problematic_fracs = [grouped_exposure[p]["harmful"] / ann_ads_count[p] for p in active_pids if ann_ads_count[p] >= 100]
print(f'fraction problematic median: {np.nanmedian(problematic_fracs):.2f}')

fraction problematic median: 0.09


### Deep Dive into Top Consumers' surveys and targeting

In [139]:
def deepdive(pid, code):
    # Show ads given to PID for code, and a summary of their survey responses for these ads
    perceptions = defaultdict(list)
    perceptions_by_ad = defaultdict(list)

    filtered = survey_codes[(survey_codes['pid'] == pid) & (survey_codes.codes.str.contains(code))]
    print(f'Total: {filtered.shape[0]}\n')

    for i in range(filtered.shape[0]):    
        row = filtered.iloc[i]
        if 'Study' in row['codes']:
            continue

        pid, adid = row['pid'], row['adid']
        # Show how many times an ad was shown
        print(adid, adid_codes[adid], int(pid_adid_freqs.loc[str(pid), str(adid)]))

        perceptions_by_ad[adid].extend([short_names[r] for r in row['like'].split(';')])
        perceptions_by_ad[adid].extend([short_names[r] for r in row['dislike'].split(';')])
        for curr_code in row['codes'].split(';'):        
            perceptions[curr_code].extend(['like: ' + short_names[r] for r in row['like'].split(';')])
            perceptions[curr_code].extend(['dislike: ' + short_names[r] for r in row['dislike'].split(';')])

    print('\n', dict(Counter(perceptions[code])))
    print('\n', dict(perceptions_by_ad))

## Things that went in paper's case studies section

### Financial Discussion (P10)

In [244]:
# experimenting if prohibited can be used to discuss this person's ads
deepdive('597658', 'Potentially Prohibited')

Total: 8

23849698645820160 Clickbait;Potentially Prohibited 1
23850177766620456 Clickbait;Potentially Prohibited 1
23849955907630513 Clickbait;Potentially Prohibited 1
23849279664790753 Clickbait;Potentially Harmful;Potentially Prohibited 1
23849536477620166 Clickbait;Potentially Prohibited 1
23849562033080494 Clickbait;Potentially Prohibited 1
23850261965810103 Clickbait;Potentially Prohibited 1
23849899280210077 Clickbait;Potentially Prohibited 1

 {'like: like-design': 1, 'dislike: dont-dislike': 3, 'like: dont-like': 5, 'dislike: pushy': 1, 'dislike: dislike-product': 1, 'dislike: scam': 4, 'like: trust-advertiser': 1, 'like: filterable': 1}

 {23849698645820160: ['like-design', 'dont-dislike'], 23850177766620456: ['dont-like', 'pushy'], 23849955907630513: ['dont-like', 'dislike-product', 'scam'], 23849279664790753: ['dont-like', 'scam'], 23849536477620166: ['dont-like', 'dont-dislike'], 23849562033080494: ['trust-advertiser', 'dont-dislike'], 23850261965810103: ['filterable', 'sc

In [237]:
deepdive('597658', 'Financial')

Total: 19

23848712731770341 Financial 1
23849228804160545 Financial 2
23849792021390405 Financial 1
23849702077860005 Financial 2
23849849948350660 Financial 1
23850646027640338 Financial 2
23849034449360137 Financial 1
23849468973880301 Financial 1
6277641471876 Financial 1
23849951550030472 Financial 1
6244444798848 Financial 6
23849625973630339 Financial 1
23849183497570702 Clickbait;Financial 1
23849794598040064 Financial 2
6269962038404 Financial 1
23849800318070670 Financial 1
23849775630970637 Financial 1
23849501078080264 Financial 1
6252731197448 Financial 1

 {'like: dont-like': 10, 'dislike: scam': 4, 'like: useful': 8, 'dislike: dont-dislike': 12, 'like: interested': 3, 'like: trust-ad': 1, 'dislike: pushy': 1, 'dislike: dislike-advertiser': 1, 'dislike: dislike-design': 1, 'like: like-design': 1}

 {23848712731770341: ['dont-like', 'scam'], 23849228804160545: ['useful', 'dont-dislike'], 23849792021390405: ['useful', 'interested', 'dont-dislike'], 23849702077860005: ['usef

In [242]:
"""
% 23849228804160545 is liked debit card ad, 23849792021390405 is liked credit monitoring ad
% >>>>>>>>>>
% $100 per day ad: 23850177766620456, opportun credit card ad: 23849183497570702, playable NFTs: 23849951550030472
"""
# FNBO card
# print(params_df.loc[23849702077860005]['WAISTUIInterestsType'])
# for i in [6002927007962, 6003061829407, 6003297396138, 6003090714101, 6003100693640, 6003130044117, 6003176678152, 6003304473660, 6003143720966, 6003142505790, 6003388314512, 6003369782940, 6003217093576, 6003154043305, 6003156575084, 6003174867249, 6003198347952, 6003199112065, 6003232036485, 6003264746114, 6003280248159, 6003466585319, 6003349442621, 6003349860951, 6003353637860, 6003402305839, 6003418314031, 6003573036687, 6003574304918, 6003716669862, 6003899195666, 6004925672432]:
#     print(interests[str(i)])
    
# # SoFi credit monitoring ad
# print('='*20)
# print(params_df.loc[23849792021390405])

# # $100 per day
# print('='*20)
# print(params_df.loc[23849183497570702])
# print(interests['6003269553527'], interests['6002998995844'])

# # opportun credit card
# print('='*20)
# cc_interests = params_df.loc[23849183497570702]['WAISTUIInterestsType']
# print(cc_interests)
# print(interests['6002927007962'])
# print(interests['6003143720966'])
# print(interests['6003369782940'])
# print(interests['6003174867249'])
# print(interests['6003198347952'])

# # NFTs
# print('='*20)
# print(params_df.loc[23849951550030472]['WAISTUIInterestsType'])
# for i in ['6003380576181', '6003540150873', '6003717913546']:
#     print(interests[i])

'\n% 23849228804160545 is liked debit card ad, 23849792021390405 is liked credit monitoring ad\n% >>>>>>>>>>\n% $100 per day ad: 23850177766620456, opportun credit card ad: 23849183497570702, playable NFTs: 23849951550030472\n'

In [239]:
deepdive('597658', 'Potentially Harmful')

Total: 5

23849279664790753 Clickbait;Potentially Harmful;Potentially Prohibited 1
23849302932210764 Potentially Harmful 1
23849360824750679 Potentially Harmful 1
6296561867328 Potentially Harmful 1
23849080851220724 Potentially Harmful 1

 {'like: dont-like': 5, 'dislike: scam': 5}

 {23849279664790753: ['dont-like', 'scam'], 23849302932210764: ['dont-like', 'scam'], 23849360824750679: ['dont-like', 'scam'], 6296561867328: ['dont-like', 'scam'], 23849080851220724: ['dont-like', 'scam']}


### Clickbait discussion (P3)

In [241]:
deepdive('805102', 'Clickbait')

Total: 16

23849737748430308 Clickbait;Potentially Prohibited 1
23849320338890245 Clickbait 1
23849448076960130 Clickbait 1
23849615540910340 Clickbait 1
23849588644970447 Clickbait;Financial 1
23849481414920452 Clickbait 1
23849432482720130 Clickbait 1
23849545267470268 Clickbait;Potentially Harmful 1
23850066375350415 Clickbait 2
23850066375350415 Clickbait 2
23850600707080459 Clickbait 1
23849954386110545 Clickbait 1
23850077806380545 Clickbait 1
23850057887360545 Clickbait 1
23849360755590643 Clickbait 1
23850246892490133 Clickbait;Healthcare;Potentially Prohibited;Sensitive 1

 {'like: interested': 5, 'like: like-design': 3, 'like: useful': 1, 'like: filterable': 15, 'dislike: unclear': 5, 'dislike: dont-dislike': 2, 'like: dont-like': 9, 'dislike: clickbait': 9, 'dislike: irrelevant': 9, 'dislike: scam': 10, 'dislike: dislike-design': 5, 'dislike: dislike-product': 4, 'like: clear': 5, 'dislike: dislike-advertiser': 2, 'dislike: political': 2, 'dislike: uncomfortable': 1}

 {2384

In [249]:
# params_df.loc[23850600707080459]['location']    # Houston, TX

# monthly assistance ad
print(params_df.loc[23850057887360545])

age_min                                         38.0
age_max                                         50.0
gender                                           NaN
location                                         NaN
WAISTUIInterestsType                             NaN
WAISTUILocaleType                                NaN
estimate_mau                              43200000.0
WAISTUICustomAudienceType                        NaN
WAISTUIBCTType                                   NaN
WAISTUIEduStatusType                             NaN
WAISTUIDPAType                                   NaN
WAISTUIActionableInsightsType                    NaN
WAISTUIFriendsOfConnectionType                   NaN
WAISTUIRelationshipType                          NaN
WAISTUIWorkEmployerType                          NaN
WAISTUILocalReachType                            NaN
WAISTUIEduSchoolsType                            NaN
WAISTUIConnectionType                            NaN
WAISTUICollaborativeAdsStoreVisitsType        

**Interesting Insight from participant 805102:** They describe Ad ID 23849737748430308 (intermittent fasting app Simple) as many good things and no dislike. On the contrary, they describe 23850246892490133 (Menopause Hormonal Belly) as irrelevant, unclear, scam, uncomfortable -- many many bad things. This shows even when someone might like something in moderation, you can go too far. People's perceptions of problematic ads can be pushed. 

Largely, they describe many of their ads with scam and dont-like, yet they see them more often than others.

### TODO: Sensitive Discussion (P1)

In [250]:
deepdive('5e5ab88a43f33029b119a48c', 'Sensitive')
# CONSIDER WRITING ABOUT THIS PERSON IN CASE STUDIES

Total: 9

23849352724070219 Healthcare;Potentially Harmful;Sensitive 1
23850858730460491 Healthcare;Sensitive 1
23849224470490027 Healthcare;Sensitive 1
23848845113870139 Healthcare;Sensitive 1
23850858630200491 Healthcare;Sensitive 1
23848927953240002 Healthcare;Sensitive 1
23850462777790491 Healthcare;Sensitive 1
23848870491900068 Healthcare;Sensitive 2
23848967546390166 Sensitive 2

 {'like: dont-like': 7, 'dislike: scam': 6, 'dislike: unclear': 2, 'like: interested': 2, 'dislike: dont-dislike': 2, 'dislike: dislike-product': 1}

 {23849352724070219: ['dont-like', 'scam'], 23850858730460491: ['dont-like', 'unclear', 'scam'], 23849224470490027: ['interested', 'dont-dislike'], 23848845113870139: ['dont-like', 'scam'], 23850858630200491: ['dont-like', 'unclear', 'scam'], 23848927953240002: ['interested', 'dont-dislike'], 23850462777790491: ['dont-like', 'scam'], 23848870491900068: ['dont-like', 'scam'], 23848967546390166: ['dont-like', 'dislike-product']}


In [256]:
# coolsculpting
params_df.loc[23849224470490027]['location']

# noom weight
# params_df.loc[23850858630200491]

"{'serialized': {'location_granularity': 'city', 'location_geo_type': 'home', 'location_code': '2463146'}, 'location_name': 'Severn, Maryland', 'location_type': 'HOME'}"

#### Older Deepdives

In [215]:
deepdive('673911', 'Clickbait')
# they like gambling -- can be seen by looking at Sensitive
# but also cannot decide when they are shown a scam!! (e.g. 23849477917770348)
# but this leads to other bad ads too

Total: 11

23849477917770348 Clickbait;Potentially Harmful 3
6279066532750 Clickbait 1
23850291251740504 Clickbait 1
23846100427010367 Clickbait 1
23849765860040469 Clickbait 1
23849678638200059 Clickbait 1
6291616560416 Clickbait 1
23848253012220775 Clickbait 1
23849552786840507 Clickbait;Financial;Potentially Prohibited 1
6271650884484 Clickbait;Financial 1
23849368496470571 Clickbait 1

 {'like: trust-advertiser': 2, 'like: trust-ad': 6, 'dislike: dont-dislike': 9, 'like: dont-like': 2, 'dislike: scam': 1, 'like: filterable': 1, 'like: like-design': 1, 'dislike: clickbait': 1, 'like: useful': 2, 'like: interested': 1}

 {23849477917770348: ['trust-advertiser', 'trust-ad', 'dont-dislike'], 6279066532750: ['dont-like', 'scam'], 23850291251740504: ['filterable', 'dont-dislike'], 23846100427010367: ['like-design', 'dont-dislike'], 23849765860040469: ['dont-like', 'clickbait'], 23849678638200059: ['trust-advertiser', 'trust-ad', 'dont-dislike'], 6291616560416: ['useful', 'dont-dislike'],

In [216]:
# 2nd clickbait person, all the remaining columns are zero but they see a lot of clickbait
deepdive('6130e3db169db34eb3cde8cc', 'Clickbait')

Total: 14

23849239160330534 Benign;Clickbait 1
23848858845620097 Benign;Clickbait 1
23848938185080519 Benign;Clickbait 1
23848957132090252 Benign;Clickbait 1
23848680911740164 Benign;Clickbait 1
23848516316660494 Benign;Clickbait 1
23849291472240012 Benign;Clickbait 1
23849465207210065 Benign;Clickbait 1
23849224532490625 Benign;Clickbait 1
23848411710970512 Benign;Clickbait 1
23849865598020133 Benign;Clickbait 1
23848915793700774 Benign;Clickbait 1
23848783775700275 Clickbait 1
23848879267500336 Benign;Clickbait 1

 {'like: dont-like': 9, 'dislike: clickbait': 10, 'dislike: irrelevant': 5, 'like: useful': 3, 'dislike: uncomfortable': 1, 'dislike: unclear': 4, 'like: amusing': 2, 'dislike: dont-dislike': 3, 'like: interested': 1, 'dislike: dislike-design': 2}

 {23849239160330534: ['dont-like', 'clickbait', 'irrelevant'], 23848858845620097: ['useful', 'clickbait', 'uncomfortable'], 23848938185080519: ['useful', 'clickbait'], 23848957132090252: ['dont-like', 'clickbait', 'irrelevant', 